# Implementing a Simple Thermostat

## Learning Objectives

- Learn the basic tools in `python-controls` library to implement basic LTI systems
- Implement an appropriate feedback system to a thermostat to achieve desired temperature


In [ ]:
import control as ctl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

## Part 1: Getting familiar with python-controls

'python-controls' is an important library in python which can be used to create mathematical models of systems, whether it be from data or from differential equations. In this course we will only be working with tools that are familiar to you theoretically. Additonally, make sure you run all of the image cells to get a better image of the problem.

Let us start by writing down a transfer function and then extracting the poles and zeros form it. The transfer function used here will be:

$$
H(s) = \frac{s + 3}{s^2 + 2s + 1}
$$

In [ ]:
# Define a transfer function
numerator = [1, 3] # coefficients of numerator 
denominator = [1, 1, 2] # coefficients of denominator
H = ctl.TransferFunction(numerator, denominator)

# Extracting poles 
poles = ctl.pole(H) # Returns array with the complex values of poles 

#Extracting zeros
zeros = ctl.zero(H) # Returns array with the complex values of zeros

print(H)
print("Poles: ", poles)
print("Zeros: ", zeros)

**Exercise 1.1:** Let us try to make a function which plots any transfer function on the s-plane. Try this function out with $H(s)$.

In [ ]:
def s_plane(tf):
    
    """
    Visualizes the poles and zeros of a given transfer function in the s-plane.

    The s-plane is a complex frequency plane, with the real part of a complex number
    plotted along the x-axis and the imaginary part along the y-axis.

    Parameters:
    - tf : Transfer Function
        The transfer function whose poles and zeros are to be extracted and visualized.
        

    Returns:
    - None : This function does not return any value.

    """
    
    poles = ... #write the function to extract the poles from a transfer function
    zeros = ... #write the function to extract the zeros from a transfer function
    
    plt.figure(figsize=(8, 8))
    plt.scatter(poles.real, poles.imag, marker='x', color='r', label='Poles')
    plt.scatter(zeros.real, zeros.imag, marker='o', color='b', label='Zeros')
    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.grid()
    plt.legend()
    plt.title('Poles and Zeros in the s-plane')
    plt.show()
    
...
    

**Exercise 1.2:** Write a function to check the stability of the system. Check if your function works with $H(s)$.

In [ ]:
def check_stability(tf):
    
    """
    Checks the stability of a given transfer function.

    Stability Criterion:
    - A transfer function is considered stable if all the poles of the transfer function
      have negative real parts (i.e., they lie in the left-half of the s-plane).

    Parameters:
    - tf : Transfer Function

    Returns:
    - None : This function prints out the stability status of the transfer function.

    """
    ...
        
check_stability(H)           

**Exercise 1.3:** Now try to analyze the stability of a system with the differential equation:

$$
\frac{d^2 y}{dt^2} - 2\frac{dy}{dt} + y(t) = x(t) 
$$

We will add snippets of code to plot the bode plot, impulse, and step response of the system.

In [ ]:
Image(filename='img1.png')

In [ ]:
G = ctl.TransferFunction(...) #Identify what the numerators and denominators will be for the following Differential Equation
...

# Plot bode plot
ctl.bode_plot(G, dB=True)

# Plot impulse response
time, response = ctl.impulse_response(G)
plt.figure()
plt.plot(time, response)
plt.title('Impulse Response')
plt.xlabel('Time')
plt.ylabel('Response')
plt.grid(True)

# Plot step response
time, response = ctl.step_response(G)
plt.figure()
plt.plot(time, response)
plt.title('Step Response')
plt.xlabel('Time')
plt.ylabel('Response')
plt.grid(True)

plt.show()


Notice how the responses of these transfers fucntions tend to infinity as time increases, further implying that the system is unstable.

In [ ]:
Image(filename='img2.png')

**Exercise 1.4:** We can introduce a feedback system to make this system stable. Let us introduce a system U(s) and compute the closed loop transfer function of the system.

$$
U(s) = {As + B}
$$

Try to find feasible values of A and B for which the transfer function will become stable. **Hint:** Think about the formula for a closed loop transfer function. 
Furthermore, use the snippets from above to get the bode plot, step response, and impulse response of the feedback system.

In [ ]:
# Define U(s)
numerator = ...
denominator = ...
U = ctl.TransferFunction(...)

# Create the closed-loop system
sys = ctl.feedback(G, U, sign = -1) # G -> Open loop TF; U -> Feedback Loop; sign -> positive or negative feedback
print(sys)
check_stability(sys)


This shows that on choosing the right feedback system, we can improve the stability of a system.

Fun Fact: As you may have noticed earlier, if the system were not stable, the phase plot will cross the -180 degree line at frequencies where the system is unstable, indicating a phase shift that can lead to positive feedback and growing oscillations.

## Part 2: Temperature Control System

You are a control systems engineering intern at a startup company called Cozy Homes. They are working on making a smart thermostat. However, on testing their device they notice that the thermostat does not provide optimal temperature because of a suspected error in the control algorithm of the thermostat. 

It is your responsibility to correct this system using your knowledge on feedback systems. You are provided with the poles and zeros of the original system in the s-plane, as given below. Furthermore, to improve the output, you decide to add a PID controller, which is a very useful type of controller. It is your goal to 'tune' the parameters of this controller to give a desirable output.

In [ ]:
Image(filename='img4.png')

**Exercise 2.1:** Load the image to get the s-plane of the overall system. From the s-plane, can you determine the transfer function of the system? You may assume the numerator of the transfer function of the system to have a constant of **0.4**.

In [ ]:
Image(filename='img3.png')

In [ ]:
gs = ctl.TransferFunction(...)

**Exercise 2.2:** We want to set the temperature on this thermostat to be 23 degrees Celsius. Feed the input to the system and plot the output of the response with respect to the input. The input is essentially a step-function with a magnitude of 23.

In [ ]:
time, response = ... # Step Response of magnitude 23
plt.figure()
# This line is to show the step input in the graph
t = np.linspace(0, 10, response.size) 
step = np.ones_like(t) * ... # Your step input

# Plotting Response
plt.plot(time, response, label='Step Response')
plt.plot(time, step, label='Step Input')
plt.title('Step Response')
plt.xlabel('Time')
plt.ylabel('Response')
plt.grid(True)
plt.legend()
plt.show()

**Exercise 2.3:** Add a unit feedback line to the system and check the output of the system. Keep in mind that a unit feedback like is just a feedback system with the transfer function:
$$
H(s) = 1
$$

In [ ]:
fb = ctl.TransferFunction(...) # Feedback Transfer Function
sys = ... # Closed Loop Transfer Function
time, response = ... # Step Response of magnitude 23

# This line is to show the step input in the graph
t = np.linspace(0, 10, response.size)
step = np.ones_like(t) * ... # Your step input

# Plotting Response 
plt.plot(time, response, label='Step Response')
plt.plot(time, step, label='Step Input')
plt.title('Step Response')
plt.xlabel('Time')
plt.ylabel('Response')
plt.legend()
plt.grid(True)


**Exercise 2.4:** Our ouput seems to approach a stady state value which is not the desired value. To optimize this response, you decide to add a controller to the system represented as the following: 

$$
C(s) = K_p + \frac{K_i}{s} + K_d s
$$


Here, C(s) is a PID controller, as mentioned earlier. The three parameters we have are $K_p$, $K_i$, and $K_d$ where:

- $K_p$ represents the proportional constant
- $K_i$ represents the integral constant
- $K_d$ represents the derivative constant 

Can you think of why the transfer function of the PID controller is so? (**Hint:** What is the Laplace transform of the integral and derivative of a function?)

For this part of the lab, cascade the following controller to the open loop system and figure out the value of the constants through a procedure of trail and error. You also find out the following relation for the constants:

$$
0 < K_p, K_i, K_d < 2
$$

The photo below is a block-diagram representation of the PID controller ($C(s)$ in the block diagram).
This is somthing you will learn more in-depth about in your future classes.

In [ ]:
Image(filename='img5.png')

In [ ]:
cs = ctl.TransferFunction(...)
gs_new = ... # This line is used to cascade two systems in series
sys = ... # Closed Loop Transfer Function
time, response = ... # Step Response of magnitude 23
plt.figure()

# This line is to show the step input in the graph
t = np.linspace(0, 10, response.size)
step = np.ones_like(t) * ...

# Plotting Response 
plt.plot(time, response, label='Step Response')
plt.plot(time, step, label='Step Input')
plt.title('Step Response')
plt.xlabel('Time')
plt.ylabel('Response')
plt.legend()
plt.grid(True)